<a href="https://colab.research.google.com/github/Pontes-y/fiap/blob/main/global_solution_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install azure-cosmos

In [174]:
import json
import secrets
import random
from google.colab import userdata
from azure.cosmos import CosmosClient, exceptions, PartitionKey

In [163]:
cosmos_endpoint = userdata.get('cosmos_endpoint')
cosmos_key = userdata.get('cosmos_key')

In [176]:
# dados do banco de pacientes
# desabilitado devido a limites da licenca da Azure Students
# p_db_name = "cosmos0"
# p_db_container = "pacientes"
# p_db_partitionkey = "grupoA"

In [165]:
# carregando arquivos de testes de pacientes
file = open('/content/test.json', 'r')
json_data = json.load(file)

In [166]:
# carregando arquivos com informações dos bancos de dados que serão utilizados para o sharding
file = open('/content/database_list.json', 'r')
database_list = json.load(file)
database_id_list = []
quantidade_db = 0
for database in database_list["databases"]:
  database_id_list.append(database["database_id"])
  quantidade_db = quantidade_db + 1

print(database_id_list)
print(quantidade_db)

[1, 2]
2


In [167]:
# função para gerar 4 bytes aleatórios (8 dígitos hexadecimais)
def gerar_numero_hexadecimal():
    numero_hexadecimal = secrets.token_hex(4)
    return numero_hexadecimal

In [168]:
# função para escolher aleatoriamente um dos banco de dados da lista
def escolher_database(lista):
  return random.choice(lista)

In [169]:
# função para gerar as informações do database escolhido
def database_config(index):
  for database in database_list["databases"]:
    if database["database_id"] == index:
      for config in database["config"]:
        database_name = config["name"]
        container_name = config["container"]
        partition_key = config["partitionKey"]
  return database_name, container_name, partition_key

In [170]:
def upload_cosmos(database_name, container_name, arquivo):
  client = CosmosClient(cosmos_endpoint, cosmos_key)

  # Criar ou obter um banco de dados
  database = client.create_database_if_not_exists(id=database_name)

  # Criar ou obter um contêiner
  container = database.create_container_if_not_exists(
    id=container_name,
    partition_key=PartitionKey(path="/chave_de_particao")
  )

  # efetua o upload
  container.upsert_item(body=arquivo)
  print("upload finalizado")

In [171]:
def upload_dynamo(database_name, container_name, arquivo):
  print("função teste")

In [177]:
# função para identificar a cloud
def upload_shard(database_name, container_name, arquivo):
  # verifica se o banco de dados é CosmosDB da Azure
  print(database_name[:6])
  if database_name[:6] == "cosmos":
    upload_cosmos(database_name, container_name, arquivo)

  # verifica se o banco de dados é DynamnoDB da AWS
  if database_name[:7] == "dyanamo":
    upload_dynamo(database_name, container_name, arquivo)

In [175]:
# Iterar sobre os pacientes no JSON
for paciente in json_data["pacientes"]:
  # cria lista em banco para adicionar informacoes do paciente
  paciente_data = {}
  quantidade_exames = 0

  # escolha aleatoria de database inicial
  database_index = escolher_database(database_id_list)
  print()

  # carrega configurações do database
  database_name, container_name, partition_key = database_config(database_index)

  # cria chave inicial
  chave_inicial = f"{database_name}-{container_name}-{partition_key}-{gerar_numero_hexadecimal()}"

  # adiciona chave inicial e informacoes necessarias para o banco de pacientes
  paciente_data["id"] = str(paciente["paciente_id"])
  paciente_data["paciente_id"] = paciente["paciente_id"]
  paciente_data["chave_inicial"] = chave_inicial
  paciente_data["chave_de_particao"] = partition_key


  # escreve paciente e chave inicial no banco de pacientes
  # upload_shard(p_db_name, p_db_container, paciente_data)

  # inicia o sharding dos exames do paciente
  for exame in paciente["exames"]:
    exame["id"] = str(exame["exame_id"])
    exame["chave_inicial"] = chave_inicial
    database_index = database_index + 1
    if database_index > quantidade_db:
      database_index = 1

    # cria chave final
    database_name, container_name, partition_key = database_config(database_index)
    chave_final = f"{database_name}-{container_name}-{partition_key}-{gerar_numero_hexadecimal()}"
    exame["chave_de_particao"] = partition_key
    exame["chave_final"] = chave_final
    chave_inicial = chave_final

    # atualiza quantidade de exames
    quantidade_exames = quantidade_exames + 1
    print(exame)

    upload_cosmos(database_name, container_name, exame)

  paciente_data["chave_final"] = chave_final
  paciente_data["quantidade_exames"] = quantidade_exames
  print(paciente_data)


{'exame_id': 1, 'resultados': False, 'id': '1', 'chave_inicial': 'cosmos1-gs2-grupoA-4ca6d58f', 'chave_de_particao': 'grupoA', 'chave_final': 'cosmos1-gs1-grupoA-2ea50ecb'}
upload finalizado
{'exame_id': 2, 'resultados': True, 'medicamentos': '1ml', 'id': '2', 'chave_inicial': 'cosmos1-gs1-grupoA-2ea50ecb', 'chave_de_particao': 'grupoA', 'chave_final': 'cosmos1-gs2-grupoA-6d16be85'}
upload finalizado
{'id': '1', 'paciente_id': 1, 'chave_inicial': 'cosmos1-gs2-grupoA-4ca6d58f', 'chave_de_particao': 'grupoA', 'chave_final': 'cosmos1-gs2-grupoA-6d16be85', 'quantidade_exames': 2}

{'exame_id': 3, 'resultados': False, 'id': '3', 'chave_inicial': 'cosmos1-gs1-grupoA-7d5e5897', 'chave_de_particao': 'grupoA', 'chave_final': 'cosmos1-gs2-grupoA-9f9148ea'}
upload finalizado
{'id': '2', 'paciente_id': 2, 'chave_inicial': 'cosmos1-gs1-grupoA-7d5e5897', 'chave_de_particao': 'grupoA', 'chave_final': 'cosmos1-gs2-grupoA-9f9148ea', 'quantidade_exames': 1}

{'exame_id': 4, 'resultados': True, 'observa